## Imports and functions

In [ ]:
from pycalphad import Model, Database, Workspace, variables as v
from pycalphad.property_framework.metaproperties import IsolatedPhase

import numpy as np

import scipy.integrate as spi
import scipy.optimize as opt
from scipy import integrate as spi

from sympy import symbols, preorder_traversal
import sympy as sp

import plotly.graph_objects as go
import matplotlib.pyplot as plt

from itertools import combinations

from symbolic_hulls_func_aggr import *

In [ ]:
def build_polynomial_basis(x, y, n):
    """
    Returns a list of Sympy monomials
    """
    basis = [1]
    for i in range(1, n+1):
        basis.append(x**i)
    for i in range(1, n+1):
        basis.append(y**i)

    return basis

def double_integral(sym_expression):
    """
    Numerically compute the integral of sym_expression(x,y) 
    """
    # Convert the sympy expression to a numeric Python function f_num(x,y).
    f_num = sp.lambdify((x, y), sym_expression, 'numpy')
    
    def integrand(yval, xval):
        return f_num(xval, yval)
    
    val, err = spi.dblquad(
        integrand,
        0,   # x-lower
        1,   # x-upper
        lambda xval: 0,  # y-lower
        lambda xval: 1 - xval  # y-upper; ternaty system
    )
    return val

def fit_ternary_polynomial_galerkin(f_expr, n):
    """
    Fits a 2D polynomial p(x,y) = sum_{i=0..n, j=0..n} alpha_{i,j} * x^i y^j
    to the given Sympy expression f_expr(x,y), via
    least-squares on [0,1]^2. Returns the symbolic polynomial approximation.
    """
    # Build basis with cross terms
    basis = build_polynomial_basis(x, y, n)
    n_basis = len(basis)  # = (n+1)^2

    # Construct the Gram matrix M and vector b
    M = sp.zeros(n_basis, n_basis)
    b_vec = sp.zeros(n_basis, 1)

    for i in range(n_basis):
        for j in range(n_basis):
            M[i, j] = double_integral(basis[i]*basis[j])
        b_vec[i, 0] = double_integral(f_expr*basis[i])

    # Solve the linear system M alpha = b
    alpha = M.LUsolve(b_vec)

    # Build the final polynomial
    p_approx = sum(alpha[i,0] * basis[i] for i in range(n_basis))
    p_approx_simple = sp.simplify(p_approx)
    
    return p_approx_simple

In [ ]:
# def compute_boundary_points(boundary, mesh_density):
#     '''
#     Compute the boundary points for the given boundary equation
#     '''
#     # Generate the 1D mesh
#     space = np.linspace(0, 1, mesh_density)
#     x, y = symbols('x y')

#     # Solve boundary=0 for y (treating x as the independent variable)
#     solutions_y = sp.solve(sp.Eq(boundary, 0), y)
#     # Solve boundary=0 for x (treating y as the independent variable)
#     solutions_x = sp.solve(sp.Eq(boundary, 0), x)
    
#     # Convert each Sympy solution to a NumPy-callable function
#     sol_y_lambdas = [sp.lambdify(x, sol, 'numpy') for sol in solutions_y]
#     sol_x_lambdas = [sp.lambdify(y, sol, 'numpy') for sol in solutions_x]

#     if sol_y_lambdas == [] and sol_x_lambdas == []:
#         return np.array([]), np.array([])

#     # We'll accumulate all x-values and y-values in lists and concatenate once.
#     x_vals = []
#     y_vals = []
    
#     # Evaluate y = f(x) for each solution
#     for f_y in sol_y_lambdas:
#         x_vals.append(space)
#         y_vals.append(f_y(space))
    
#     # Evaluate x = f(y) for each solution
#     for f_x in sol_x_lambdas:
#         x_vals.append(f_x(space))
#         y_vals.append(space)
    
#     x_vals = np.array(x_vals)
#     y_vals = np.array(y_vals)
#     for arr in [x_vals, y_vals]:
#         if arr.ndim == 0:
#             arr = np.expand_dims(arr, axis=0)

#     # Combine into final arrays
#     X = np.concatenate(x_vals)
#     Y = np.concatenate(y_vals)
    
#     # Apply the mask: only keep points where X + Y <= 1
#     # Replace invalid points with np.nan (or you could filter them out)
#     mask = ((X + Y) <= 1) & (X >= 0) & (Y >= 0)
#     X = X[mask]
#     Y = Y[mask]
    
#     return X, Y

In [ ]:
def compute_boundary_points(boundary, mesh_density):
    '''
    Compute the boundary points for the given boundary equation.
    '''
    # Generate the 1D mesh
    space = np.linspace(0, 1, mesh_density)
    x, y = symbols('x y')

    # Solve boundary=0 for y (treating x as the independent variable)
    solutions_y = sp.solve(sp.Eq(boundary, 0), y)
    # Solve boundary=0 for x (treating y as the independent variable)
    solutions_x = sp.solve(sp.Eq(boundary, 0), x)
    
    # Convert each Sympy solution to a NumPy-callable function
    sol_y_lambdas = [sp.lambdify(x, sol, 'numpy') for sol in solutions_y]
    sol_x_lambdas = [sp.lambdify(y, sol, 'numpy') for sol in solutions_x]

    # If no solutions, return empty arrays
    if not sol_y_lambdas and not sol_x_lambdas:
        return np.array([]), np.array([])

    x_vals = []
    y_vals = []
    
    # Evaluate y = f(x) for each solution
    for f_y in sol_y_lambdas:
        x_vals.append(space)
        y_vals.append(f_y(space))
    
    # Evaluate x = f(y) for each solution
    for f_x in sol_x_lambdas:
        x_vals.append(f_x(space))
        y_vals.append(space)
    
    # Ensure each sub-array is at least 1D before concatenating
    x_vals = [np.atleast_1d(arr) for arr in x_vals]
    y_vals = [np.atleast_1d(arr) for arr in y_vals]
    
    # Now it's safe to concatenate
    X = np.concatenate(x_vals)
    Y = np.concatenate(y_vals)
    
    # Apply the mask: only keep points where X + Y <= 1, and both X, Y >= 0
    mask = ((X + Y) <= 1) & (X >= 0) & (Y >= 0)
    X, Y = np.where(mask, X, np.nan), np.where(mask, Y, np.nan)
    
    return X, Y

## Real system

In [ ]:
# Teperature
temperature = 1000

# Variable dictionary here
x, y = symbols('x y')
symbol_dict = {
    'CU': x,
    'CU2': 2*x,
    'AL': y,
    'Y': 1-x-y,
    'VA': 1}

# Polynomial order
n = 2

In [ ]:
db = Database("../TDDatabaseFiles_temp/ALCUY-2011.TDB")
phases = list(db.phases.keys())
# phases = ['AL42CU68Y10', 'AL7CU2Y3']    # Test phases
# phases = ['CU6Y']
print(phases)

In [ ]:
# Create a dictionary to store the polynomial expressions
phase_energy_expression_dict = dict()

# First we need to go through all phases and fit polynomials to their energy functions
for phase in phases:
    comps = [list(sublattice)[0].name for sublattice in db.phases[phase].constituents]
    model = Model(db, comps, phase)

    sp_energy_expr = sp.sympify(model.GM)  # Get the Gibbs energy expression in sympy

    # Get the symbols from the non-converted expression
    chem_symbols = model.GM.free_symbols
    chem_symbols.remove(v.T)

    # Evaluate the function at some temperature
    sp_energy_expr = sp_energy_expr.subs(v.T, temperature)
    for chem_symbol in chem_symbols:
        name = chem_symbol.species.name
        sp_energy_expr = sp_energy_expr.subs(chem_symbol, symbol_dict[name])

    # Fit a polynomial to the energy function
    print('Fitting polynomial to', phase)
    poly_energy_expr = fit_ternary_polynomial_galerkin(sp_energy_expr, n)

    # Round off the polynomials
    poly_energy_expr = poly_energy_expr.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))

    # Add the polynomial to the dictionary
    phase_energy_expression_dict[phase] = [poly_energy_expr, sp.lambdify((x, y), sp_energy_expr, 'numpy')]    # We store both the polynomial and the original expression as a vectorized function

In [ ]:
# This is just to visualize all real energy functions

fig = go.Figure()
X, Y = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
mask = X + Y <= 1
X, Y = np.where(mask, X, np.nan), np.where(mask, Y, np.nan)

for phase in phase_energy_expression_dict:
    energy_func = phase_energy_expression_dict[phase][1]
    Z = energy_func(X, Y)
    fig.add_trace(go.Surface(z=Z, x=X, y=Y, name=phase, showscale=False))

fig.update_layout(
    title="Energy functions", 
    scene=dict(
        aspectmode='cube',
        xaxis_title='Cu',
        yaxis_title='Al',
        zaxis_title='G'
    ),
    width=800,
    height=700,
)

fig.show()

In [ ]:
phase_pairs = list(combinations(phase_energy_expression_dict, 2))
phase_pairs_dict = dict()

for pair in phase_pairs:
    # Compute the phase boundaries
    boundary1 = hpboundry_f2_to_f1(phase_energy_expression_dict[pair[0]][0], phase_energy_expression_dict[pair[1]][0])
    boundary2 = hpboundry_f2_to_f1(phase_energy_expression_dict[pair[1]][0], phase_energy_expression_dict[pair[0]][0])

    phase_pairs_dict[pair] = boundary1
    # Reverse the phase pairs and add the other boundary
    phase_pairs_dict[(pair[1], pair[0])] = boundary2

In [ ]:
# Record the phase that a point is lying on
boundary_points_lists = dict()

# Got through all the phase pairs and compute the boundary points
mesh_density = 100
for pair in phase_pairs_dict:
    # Get the boundary points in X and Y
    Xline, Yline = compute_boundary_points(phase_pairs_dict[pair], mesh_density)

    # The polynomial isnt the best fit to the energy so we will compute actual energies off of the true function
    energy_lambda = sp.lambdify((x, y), phase_energy_expression_dict[pair[0]][0], 'numpy')
    Energies = energy_lambda(Xline, Yline)

    # Aggregate all the points
    data = np.column_stack((Xline, Yline, Energies))

    # Initialize a list for this phase if it doesn't exist yet
    if phase not in boundary_points_lists:
        boundary_points_lists[pair[0]] = []

    # Append the new data to the list
    boundary_points_lists[pair[0]].append(data)

boundary_points = {}
for phase, data_list in boundary_points_lists.items():
    boundary_points[phase] = np.vstack(data_list)

In [ ]:
# Create a composition space to plot the phases
X = np.linspace(0, 1, 100) 
Y = np.linspace(0, 1, 100)

# Create meshgrid
X, Y = np.meshgrid(X, Y)
mask = (X + Y) <= 1

# Create a triangular mask for the ternary system
X, Y = np.where(mask, X, np.nan), np.where(mask, Y, np.nan)

In [ ]:
# Plot with Plotly
fig = go.Figure()

for phase in phases:

    # Add the energy function
    fig.add_trace(
        go.Surface(
            x=X, y=Y, z=sp.lambdify((x, y), phase_energy_expression_dict[phase][0], 'numpy')(X, Y),
            colorscale='RdBu',
            opacity=0.7,
            name=phase,
            showscale=False
        )
    )

    # Add the boundary points
    fig.add_trace(
        go.Scatter3d(
            x=boundary_points[phase][:,0], y=boundary_points[phase][:,1], z=boundary_points[phase][:,2],
            mode='markers',
            marker=dict(size=5),
            name=f'{phase} boundary points'
        )
    )

fig.update_layout(
    title="Energy functions and boundaries", 
    scene=dict(
        aspectmode='cube',
        xaxis_title='Cu',
        yaxis_title='Al',
        zaxis_title='G'
    ),
    width=800,
    height=700
)

fig.show()